# Predictions

This notebook simply uses our classifier to predict the class of the test data and store the results in a CSV file to be submitted to the competition.

In [1]:
import cv2
import os

import numpy as np
import pandas as pd
import random
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt

from utils.utils import *
from utils.display import plot_images, plot_coins
from utils.localization import *
from utils.model_setup import CoinDataset

import torch
import torch.nn as nn

test_folder = 'data/test'
num_classes = 16

/home/avray/anaconda3/envs/iapr/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/avray/anaconda3/envs/iapr/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Load pre-trained efficient-net B0 model
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0')
model.classifier.fc = torch.nn.Sequential(nn.Linear(model.classifier.fc.in_features, num_classes*100),
                                        nn.BatchNorm1d(num_classes*100),
                                        nn.ReLU(),
                                        nn.Dropout(0.5),
                                        nn.Linear(num_classes*100, num_classes*50),
                                        nn.BatchNorm1d(num_classes*50),
                                        nn.ReLU(),
                                        nn.Dropout(0.5),
                                        nn.Linear(num_classes*50, num_classes*10),
                                        nn.BatchNorm1d(num_classes*10),
                                        nn.ReLU(),
                                        nn.Dropout(0.5),
                                        nn.Linear(num_classes*10, num_classes))

model.load_state_dict(torch.load('efficientb0.ckpt'))

Using cache found in /home/avray/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/avray/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/avray/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


<All keys matched successfully>

In [3]:
model.eval()


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [4]:
submission_data = pd.read_csv('data/sample_submission.csv')
submission_data

,id,5CHF,2CHF,1CHF,0.5CHF,0.2CHF,0.1CHF,0.05CHF,2EUR,1EUR,0.5EUR,0.2EUR,0.1EUR,0.05EUR,0.02EUR,0.01EUR,OOD
0,L0000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,L0000001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,L0000002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,L0000003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,L0000004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,L0000157,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
158,L0000158,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
159,L0000159,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
160,L0000160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
cnt = 0
for b in range(np.ceil(162/6).astype(int)):
    # load images 6 by 6
    images = load_images_from_folder(test_folder, max_images=6, batch_idx=6*b,resize=False)
    coins,masks,coins_coord = extract_coins(images)

    # print(masks[0][0].max())
    for n in range(len(coins)):
        y_labels = [0 for _ in range(len(coins[n]))]
        test_coins = coins[n]
        test_masks = [(masks[n][i]*255).astype(np.uint8) for i in range(len(masks[n]))]

        valid_dataset = CoinDataset(test_coins, test_masks, y_labels)#, redim_size=(380,380))
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)

        y_pred = []
        for images, labels in valid_loader:
            # Forward pass through the model
            outputs = model(images)
            
            _, predicted = torch.max(outputs, 1)
            
            y_pred.extend(predicted.cpu().numpy())

        output = np.zeros(16)
        for i in range(len(y_pred)):
            output[y_pred[i]] += 1

        print(cnt,output)
        submission_data.iloc[cnt, 1:] = output
        cnt += 1

0 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
1 [0. 1. 1. 1. 2. 0. 1. 0. 2. 1. 0. 0. 1. 0. 0. 0.]
2 [2. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
3 [0. 0. 0. 3. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 2.]
4 [0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
5 [1. 0. 2. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


KeyboardInterrupt: 

In [ ]:
submission_data

,id,5CHF,2CHF,1CHF,0.5CHF,0.2CHF,0.1CHF,0.05CHF,2EUR,1EUR,0.5EUR,0.2EUR,0.1EUR,0.05EUR,0.02EUR,0.01EUR,OOD
0,L0000000,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1
1,L0000001,0,1,1,1,2,0,1,0,2,1,0,0,1,0,0,0
2,L0000002,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,L0000003,0,0,0,3,0,0,1,0,0,0,0,1,0,1,0,2
4,L0000004,0,0,1,1,1,1,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,L0000157,0,1,0,0,1,0,0,2,0,0,0,0,0,0,0,0
158,L0000158,1,0,0,1,0,1,1,0,0,1,0,0,1,1,0,0
159,L0000159,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
160,L0000160,0,0,0,0,2,0,1,0,0,0,1,3,0,0,0,0


In [ ]:
submission_data.to_csv('data/submission.csv', index=False)